In [9]:
from datetime import datetime

datetime.now().isoformat()[:19].replace('T', '')

'2024-06-13 16:47:40'

In [54]:
import os
import json


def compare_evaluate(result_path, dataset_name, split, model_name, prompt_mode, backup, self_refine_round, output_path):
    result_file = os.path.join(result_path, f'self-refine-{self_refine_round}_{dataset_name}_{split}_{model_name}_{prompt_mode}_backup-{backup}.json')
    original_result_file = os.path.join(result_path, f'{dataset_name}_{split}_{model_name}_{prompt_mode}_backup-{backup}.json')


    with open(original_result_file, 'r') as f:
        original_results = json.load(f)

    with open(result_file, 'r') as f:
        results = json.load(f)
        
        
    original_dict = {sample['id']: sample for sample in original_results}
    results_dict = {sample['id']: sample for sample in results}
    new_executable = [sample for sample_id, sample in results_dict.items() if sample['flag']=='success' and original_dict[sample_id]['flag'] != 'success']

    if not os.path.exists(f'compared_evaluation/{output_path}'):
        os.makedirs(f'compared_evaluation/{output_path}')

    with open(f'compared_evaluation/{output_path}/self-refine-{self_refine_round}_{dataset_name}_{split}_{model_name}_{prompt_mode}_backup-{backup}.json', 'w') as f:
        json.dump(new_executable, f, indent=4)

In [57]:
result_path = 'outputs_llama3_70B/logic_inference'
dataset_name = 'FOLIO'
split = 'dev'
model_names = ['gpt-3.5-turbo', 'gpt-4-turbo', 'gpt-4o']
prompt_mode = 'dynamic'
backup = 'random'
output_paths = ['llama3_70B', 'mixtral_8x7B', 'mistral_7B']

for output_path in output_paths:
    for model_name in model_names:
        for self_refine_round in range(1, 4):
            compare_evaluate(result_path, dataset_name, split, model_name, prompt_mode, backup, self_refine_round, output_path)


In [68]:
os.path.split(result_path)

('outputs_llama3_70B', 'logic_inference')

## Count tokens

In [164]:
import tiktoken

enc = tiktoken.get_encoding('cl100k_base')

In [170]:
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding('cl100k_base')
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [176]:
import json

with open('data/LogicNLI/train.json', 'r') as f:
    data = json.load(f)
    
    total = 0
    for d in data:

        total += num_tokens_from_string(' '.join(d['context']))
        total += num_tokens_from_string(' '.join(d['context_fol']))
        total += num_tokens_from_string(d['question'])
        total += num_tokens_from_string(d['question_fol'])
    
    
with open('data/LogicNLI/dev.json', 'r') as f:
    data = json.load(f)
    
    for d in data:

        total += num_tokens_from_string(' '.join(d['context']))
        total += num_tokens_from_string(' '.join(d['context_fol']))
        total += num_tokens_from_string(d['question'])
        total += num_tokens_from_string(d['question_fol'])
    
    print(total)

8053624


In [177]:
(total/1000000)*0.13

1.04697112

## Convert LogicNLI 

In [3]:
import json

with open('data/LogicNLI_original/train_language.json', 'r') as f:
    sample_language = json.load(f)

with open('data/LogicNLI_original/train_logic.json', 'r') as f:
    sample_logic = json.load(f)


In [4]:
label_mapping = {
    'entailment': 'A',
    'contradiction': 'B',
    'self_contradiction': 'B',
    'neutral': 'C'
}

In [5]:

output = []

for story_id, ((data_id, data), (statement_id, statement)) in enumerate(zip(sample_logic.items(), sample_language.items())):
    
    context_fol = []
    context = []
    for (fact_id, fact), (fact_nl) in zip(data['facts'].items(), statement['facts']):
        
        subject = fact[0].lower()
        attribute = fact[1].capitalize()
        polarity = fact[2]
        
        context_fol.append(f"{attribute}({subject})" if polarity == '+' else f"¬{attribute}({subject})")
        context.append(fact_nl)
        
    for (rule_id, rule_data), (rule_nl) in zip(data['rules'].items(), statement['rules']):
        premise = []
        
        same_quant_subject = all(fact[0] == rule_data['p']['fact'][0][0] for fact in rule_data['p']['fact']) and all(fact[0] in ['all', 'exist'] for fact in rule_data['p']['fact'])
        
        for fact in rule_data['p']['fact']:
            
            attribute = fact[1].capitalize()
            polarity = fact[2]
            
            if same_quant_subject:
                quant = "∃x " if fact[0] == 'exist' else "∀x "
                
                premise.append(f'{attribute}(x)' if polarity == '+' else f'¬{attribute}(x)')
            
            else:
                if fact[0] == 'exist':
                    premise.append(f"∃x " + (f'{attribute}(x)' if polarity == '+' else f'¬{attribute}(x)'))
                elif fact[0] == 'all':
                    premise.append(f"∀x " + (f'{attribute}(x)' if polarity == '+' else f'¬{attribute}(x)'))
                else:
                    subject = fact[0].lower()
                    premise.append(f'{attribute}({subject})' if polarity == '+' else f'¬{attribute}({subject})')
                
        if same_quant_subject:
            premise_str = quant + '('
        else:
            premise_str = '('
        premise_str += ' ∨ '.join(premise) if rule_data['p']['conj'] == 'or' else ' ∧ '.join(premise)
        premise_str += ')'

        same_quant_subject = all(fact[0] == rule_data['q']['fact'][0][0] for fact in rule_data['q']['fact']) and all(fact[0] in ['all', 'exist'] for fact in rule_data['q']['fact'])

        conclusion = []
        for fact in rule_data['q']['fact']:
            
            attribute = fact[1].capitalize()
            polarity = fact[2]
            
            if same_quant_subject:
                quant = "∃x " if fact[0] == 'exist' else "∀x "
                
                conclusion.append(f'{attribute}(x)' if polarity == '+' else f'¬{attribute}(x)')
            
            else:
                if fact[0] == 'exist':
                    conclusion.append(f"∃x " + (f'{attribute}(x)' if polarity == '+' else f'¬{attribute}(x)'))
                elif fact[0] == 'all':
                    conclusion.append(f"∀x " + (f'{attribute}(x)' if polarity == '+' else f'¬{attribute}(x)'))
                else:
                    subject = fact[0].lower()
                    conclusion.append(f'{attribute}({subject})' if polarity == '+' else f'¬{attribute}({subject})')
                
                
        if same_quant_subject:
            conclusion_str = quant + '('
        else:
            conclusion_str = '('
        conclusion_str += ' ∨ '.join(conclusion) if rule_data['q']['conj'] == 'or' else ' ∧ '.join(conclusion)
        conclusion_str += ')'
        
        # print(conclusion_str)

        if rule_data['type'] == 'imp':
            context_fol.append(f"({premise_str}) → ({conclusion_str})")
        else:
            context_fol.append(f"({premise_str}) ↔ ({conclusion_str})")
            
        context.append(rule_nl)
    
    
    for question_id, ((_, question), (question_nl), (label)) in enumerate(zip(data['statements'].items(), statement['statements'], statement['labels'])):

        if label == 'self_contradiction':
            continue

        attribute = question[1].capitalize()
        polarity = question[2]
        subject = question[0].lower()
        
        question_fol = f'{attribute}({subject})' if polarity == '+' else f'¬{attribute}({subject})'        
        
        output.append({
            'id': int(question_id),
            'story_id': int(story_id),
            'context': context,
            'context_fol': context_fol,
            'question': question_nl,
            'question_fol': question_fol,
            'answer': label_mapping[label]
        })


In [6]:
with open('data/LogicNLI/train.json', 'w') as f:
    json.dump(output, f, indent=2, ensure_ascii=False)

In [7]:
with open('data/LogicNLI/train.json', 'r') as f:
    a = json.load(f)

## Predicates

In [7]:
import re
import json

reg = re.compile(r'(?!x|y|z|w)\b\w+\s*\([^\)]+\)')

with open('data/LogicNLI/train.json', 'r') as f:
    train = json.load(f)

In [8]:
args_map = {
    1: 'x',
    2: 'x, y',
    3: 'x, y, z',
    4: 'x, y, z, w'
}

In [9]:
def clean_predicate(predicate):
    name = predicate.split('(')[0].strip()
    arguments = [p.split() for p in predicate.split('(')[1][:-1].split(',')]
    
    n_arguments = len(arguments)
    
    clean_predicate = name + '(' + args_map[n_arguments] + ')'

    return clean_predicate

In [10]:
for point in train:
    formulas = ' '.join(point['context_fol'])
    preds = list(set(re.findall(reg, formulas)))
    
    clean_preds = list(set(clean_predicate(pred) for pred in preds))
    
    point['logic_predicates'] = clean_preds

In [11]:
with open('data/LogicNLI/train.json', 'w') as f:
    json.dump(train, f, indent=2, ensure_ascii=False)

## The rest

In [3]:
import json

In [7]:
with open('outputs/logic_predicates/FOLIO_dev_gpt-3.5-turbo.json', 'r') as f:
    preds = json.load(f)

In [8]:
with open('outputs/logic_programs/FOLIO_dev_gpt-3.5-turbo_static.json', 'r') as f:
    progs = json.load(f)

In [13]:
preds.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '113', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '1

In [17]:
for prog in progs:
    if str(prog['id']) in preds:
        prog['predicates'] = preds[str(prog['id'])]['logic_predicates']
    else:
        prog['predicates'] = []

In [18]:
progs

[{'id': 0,
  'context': ['If people perform in school talent shows often, then they attend and are very engaged with school events.',
   'People either perform in school talent shows often or are inactive and disinterested members of their community.',
   'If people chaperone high school dances, then they are not students who attend the school.',
   'All people who are inactive and disinterested members of their community chaperone high school dances.',
   'All young children and teenagers who wish to further their academic careers and educational opportunities are students who attend the school.',
   'Bonnie either both attends and is very engaged with school events and is a student who attends the school, or she neither attends and is very engaged with school events nor is a student who attends the school. '],
  'question': 'Bonnie performs in school talent shows often.',
  'answer': 'C',
  'raw_logic_programs': ['First-Order-Logic Premises:\n∀x (Perform(x) → (Attend(x) ∧ Engaged(x))

## GCD

In [11]:
with open('models/grammars/FOLIO.gbnf', 'r') as f:
    grammar = f.read()

In [13]:
user = """
Natural Language Statement: Mia's favorite season is not the same as Emma's.

Sketch First Order Logic Statement: FavoriteSeason(mia, y) ∧ y ≠ FavoriteSeason(emma, y)

Predicates:
\"\"\"
Season(x) ::: x is a season
WantlongVacation(x) ::: x wants a long vacation
Love(x,y) ::: y is the favourite season of x
\"\"\"
######
Valid FOL Statement: FavoriteSeason(mia, y) ∧ ¬(FavoriteSeason(emma, y))"
------
Natural Language Statement: All restaurants with a rating greater than 9 are listed in Yelp’s recommendations.

Sketch First Order Logic Statement: ∀x (Rating(x, y) ∧ y > 9 → Listed(x))

Predicates:
\"\"\"
Listed(x) ::: x is listed in Yelp’s recommendations
RatingGreaterThan9(x) ::: x has a rating greater than 9
NegativeReview(x) ::: x receives many negative reviews
PopularAmongLocalResidents(x) ::: x is popular among local residents
NoTakeOutService(x) ::: x has no takeout service
\"\"\"
######
Valid FOL Statement: ∀x (RatingGreaterThan9(x)→ Listed(x))
------
Natural Language Statement: When a person reads a book, that person gains knowledge.

Sketch First Order Logic Statement: ∀x ∀y (ReadBook(x, y) → Gain(Knowledge(x)))

Predicates:
\"\"\"
Book(x) ::: x is a book
Knowledge(x) ::: x contains knowledge
ReadBook(x, y) ::: x reads book y
GainKnowledge(x) ::: x gains knowledge
Smarter(x) ::: x becomes smarter
\"\"\"
######
Valid FOL Statement: ∀x ∀y (ReadBook(x, y) → GainKnowledge(x))
------
Natural Language Statement: [BG] If a game sells more than one million copies, then it will be selected into the Top 10 list.

Sketch First Order Logic Statement: ∀x (Sell(x, legendOfZelda) ∧ x > oneMillion → Select(x, Top10))

Predicates:
\"\"\"
Game(x) ::: x is a game.
Company(x, y) ::: x is a company and y is a game created by x.
Create(x, y) ::: x created the game y.
Japanese(x) ::: x is a Japanese game company.
Top10(x) ::: x is in the Top 10 list.
Sell(x, y) ::: x sells y copies.
SellMore(x, y) ::: x sells more than y copies.
Select(x, y) ::: x is selected into the Top 10 list.
\"\"\"
######
Valid FOL Statement: 
"""

In [14]:
task_description = """
Given a Natural Language Statement, a Sketch First Order Logic Statement and a set of Predicates. 
The task is to rewrite the Sketch using only the provided Predicates, and fix errors if any.
Your goal is to ensure validity and accuracy of the formulations.

Here are a few examples.
------
"""

In [17]:
raw_grammar = """
root ::= S

S ::= F | QUANT VAR (S1 | F) | "¬" (S1 | F)
S1 ::= F | QUANT VAR (S2 | F) | "¬" (S2 | F)
S2 ::= F | QUANT VAR (S3 | F) | "¬" (S3 | F)
S3 ::= F | QUANT VAR F | "¬" F

F ::= "¬"? "(" F1 ")" | F1 OP F1 | L
F1 ::= "¬" "(" F2 ")" | "(" F2 ")" | F2 OP F2 | L
F2 ::= "¬" "(" F3 ")" | "(" F3 ")" | F3 OP F3 | L
F3 ::= L 



OP ::= "⊕" | "∨" | "∧" | "→" | "↔"
L ::= "¬"? PRED "(" TERMS ")"
TERMS ::= TERM | TERM "," TERMS
TERM ::=  VAR
QUANT ::= "∀" | "∃"
VAR ::= "x" | "y" | "z"
PRED ::= "Game" | "Company" | "Create" | "Japanese" | "Top10" | "Sell" | "Select" | "SellMore"
"""

In [18]:
from llama_cpp.llama import LlamaGrammar
from llama_cpp import Llama

model_path = "GCD/llms/nous-hermes-2-solar-10.7b.Q6_K.gguf"
n_ctx = 2048
n_gpu_layers = -1
n_batch = 512

llm = Llama(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx = n_ctx,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose = False
)



In [19]:

max_tokens=50 
temperature=0.01
frequency_penalty=0.0
repeat_penalty=1.1
# presence_penalty=0.0
top_p=0.9
top_k=20
stop=['------', '###']

grammar = LlamaGrammar.from_string(raw_grammar, verbose=False)


In [20]:

result = llm.create_chat_completion(
messages=[
    {"role": "system", "content": task_description},
    {"role": "user", "content": user}
    ],
max_tokens = max_tokens,

frequency_penalty = frequency_penalty,
repeat_penalty = repeat_penalty,
# presence_penalty = presence_penalty,

temperature=temperature,

top_p=top_p,
top_k=top_k,

stop = stop,
grammar = grammar,
)

In [21]:
result['choices'][0]['message']['content']

'∀x(Sell(x,y)∧SellMore(y,x)→Select(x,y))'

## Logic

In [69]:
import re
import json
import os
from copy import deepcopy
from nltk.inference.prover9 import Prover9Command
from nltk.sem.logic import *

from models.symbolic_solvers.fol_solver.Formula_util import FOL_Formula
from models.symbolic_solvers.fol_solver.fol_prover9_parser import Prover9_FOL_Formula

from sentence_transformers import SentenceTransformer, util
import numpy as np
from tqdm.autonotebook import tqdm

os.environ['PROVER9'] = './models/symbolic_solvers/Prover9/bin'

Samples removed from dev:

- 66
- 67
- 68
- 138
- 139
- 171

Samples removed from train:

- 691

In [37]:
original = Expression.fromstring('all x.(Roundel(x) -> -Higher(x) -> Lower(x))')
new = Expression.fromstring('all x y.exists z(Know(x, z) & Know(y, z) & UniversalLanguage(z)) -> Communicate(x, y)')

original.equiv(new)

LookupError: 

===========================================================================
NLTK was unable to find the prover9 file!
Use software specific configuration parameters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on prover9, see:
    <https://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [38]:
original

<AllExpression all x.((Roundel(x) -> -Higher(x)) -> Lower(x))>

(all x.(GSCjasdhjvfd))

∀x (WantlongVacation(x) → (Love(x, summer) ∧ ¬Love(x, spring) ∧ ¬Love(x, fall) ∧ ¬Love(x, winter)))

In [116]:
context_fol= ['Angry(adler)',
 'Short(wiley)',
 'Short(rosa)',
 '¬Angry(rosa)',
 '¬Poised(rosa)',
 '¬Poised(wiley)',
 '¬Angry(wiley)',
 'Shiny(rosa)',
 '¬Different(wiley)',
 'Short(cary)',
 '¬Different(blaine)',
 'Poised(dan)',
 '(∃x (Different(x) ∨ Poised(x))) → ((Talkative(wiley)))',
 '(∀x (Talkative(x) ∨ ¬Poised(x))) → ((¬Angry(rosa)))',
 '(∀x (Short(x) ∧ ¬Shiny(x))) ↔ ((∀x ¬Angry(x)))',
 '(∀x (Different(x) ∧ Short(x))) ↔ ((∀x ¬Talkative(x)))',
 '((Shiny(dan) ∨ Different(adler))) → ((Short(wiley)))',
 '((¬Short(rosa))) → ((Angry(adler) ∧ ¬Different(blaine)))',
 '((Talkative(wiley) ∨ Short(adler))) → ((Different(wiley)))',
 '(∃x (Angry(x))) → ((¬Talkative(dan)))',
 '((Poised(dan))) ↔ ((¬Shiny(rosa)))',
 '((Poised(blaine) ∧ ¬Angry(blaine))) → ((Different(rosa) ∧ Shiny(rosa)))',
 '(∃x (¬Short(x))) → ((Talkative(adler)))',
 '(∀x (¬Shiny(x) ∨ Short(x))) → ((∀x ¬Angry(x)))']

question_fol = "TrainTextSumModel(supervisedLearning)"

In [117]:
ass= []

for s in context_fol:
    fol_formula = FOL_Formula(s)

    if fol_formula.is_valid:
        prover9_formula = Prover9_FOL_Formula(fol_formula).formula
        # 
        ass.append(Expression.fromstring(prover9_formula))
    else:
        print(f'error: ', s)

In [46]:
goal = Expression.fromstring(Prover9_FOL_Formula(FOL_Formula(question_fol)).formula)

In [118]:
ass

[<ApplicationExpression Angry(Adler)>,
 <ApplicationExpression Short(Wiley)>,
 <ApplicationExpression Short(Rosa)>,
 <NegatedExpression -Angry(Rosa)>,
 <NegatedExpression -Poised(Rosa)>,
 <NegatedExpression -Poised(Wiley)>,
 <NegatedExpression -Angry(Wiley)>,
 <ApplicationExpression Shiny(Rosa)>,
 <NegatedExpression -Different(Wiley)>,
 <ApplicationExpression Short(Cary)>,
 <NegatedExpression -Different(Blaine)>,
 <ApplicationExpression Poised(Dan)>,
 <ImpExpression (exists x.(Different(x) | Poised(x)) -> Talkative(Wiley))>,
 <ImpExpression (all x.(Talkative(x) | -Poised(x)) -> -Angry(Rosa))>,
 <IffExpression (all x.(Short(x) & -Shiny(x)) <-> all x.-Angry(x))>,
 <IffExpression (all x.(Different(x) & Short(x)) <-> all x.-Talkative(x))>,
 <ImpExpression ((Shiny(Dan) | Different(Adler)) -> Short(Wiley))>,
 <ImpExpression (-Short(Rosa) -> (Angry(Adler) & -Different(Blaine)))>,
 <ImpExpression ((Talkative(Wiley) | Short(Adler)) -> Different(Wiley))>,
 <ImpExpression (exists x.Angry(x) -> -T

In [65]:
goal

<ApplicationExpression TrainTextSumModel(SupervisedLearning)>

In [66]:
prover = Prover9Command(goal, ass)

In [61]:
goal = NegatedExpression(goal)

In [52]:
prover = Prover9Command(goal, ass)

In [67]:
prover.prove()

False

In [54]:
prover.proof(simplify=False)

'============================== Prover9 ===============================\nProver9 (64) version 2009-11A, November 2009.\nProcess 21297 was started by feder on wlan-221177.nbw.tue.nl,\nFri May 31 14:06:00 2024\nThe command was "./models/symbolic_solvers/Prover9/bin/prover9".\n============================== end of head ===========================\n\n============================== INPUT =================================\nassign(max_seconds,60).\nclear(auto_denials).\n\nformulas(assumptions).\n(all x (Mammal(x) -> Animal(x))).\n(all x (Monkey(x) -> Mammal(x))).\n(all x (Monkey(x) | Bird(x))).\n(all x (Bird(x) -> Fly(x))).\n(all x (Animal(x) -> Breathe(x))).\n(all x (Fly(x) -> HaveWings(x))).\n(-Fly(Rock) & -Bird(Rock) -> -Fly(Rock)) & -Breathe(Rock).\nend_of_list.\n\nformulas(goals).\n-Monkey(Rock).\nend_of_list.\n\n============================== end of input ==========================\n\n============================== PROCESS NON-CLAUSAL FORMULAS ==========\n\n% Formulas that are not ordin

In [5]:
ass

[<AllExpression all x.((GrandSlamChampion(x) & -OscarNominatedActor(x)) | (-GrandSlamChampion(x) & OscarNominatedActor(x)))>]

In [ ]:
with open('old/outputs/logic_inference/FOLIO_dev_gpt-3.5-turbo_static_text_backup-LLM.json', 'r') as f:
    old_inference = json.load(f)

In [ ]:
with open('outputs/logic_inference/FOLIO_dev_gpt-3.5-turbo_static_text_backup-LLM.json', 'r') as f:
    new_inference = json.load(f)

## Conversion Datasets

In [ ]:
with open('parsing/new_formulas_v0/FOLIO_dev_gpt-3.5-turbo_static_text.json', 'r') as j:
    folio_v0_train_parsed = json.load(j)

In [ ]:
with open('data/FOLIO/dev.json', 'r') as j:
    folio_v0_train = json.load(j)

In [ ]:
folio_v0_train_dict = {val['id']: val for val in folio_v0_train}

In [ ]:
folio_v0_train_parsed_dict = {val['id']: val for val in folio_v0_train_parsed}

In [ ]:
for id in folio_v0_train_dict.keys():
    folio_v0_train_dict[id]['context_fol'] = folio_v0_train_parsed_dict[id]['assumptions']
    folio_v0_train_dict[id]['question_fol'] = folio_v0_train_parsed_dict[id]['goal']

In [ ]:
new_folio_v0_train = [val for key, val in folio_v0_train_dict.items()]

In [ ]:
with open('parsed_data/FOLIO/dev.json', 'w') as j:
    json.dump(new_folio_v0_train, j)